In [1]:
import numpy as np
import pandas as pd
import import_ipynb

In [ ]:
import Orders, Freights
import Customers, PlantPorts, ProductsPerPlant, WhCapacities, WhCosts

# 2da Etapa: Optimización

*Función Objetivo*\
Se desea conocer un conjunto de plantas de almacenaje, puertos de origen y puertos de destino que minimicen el costo total de la cadena de suministro.

Nomenclatura:
- k: ID de pedido
- i: ID de planta de almacenamiento
- p: ID de puerto de origen
- j: ID de puerto de destino
- c: ID de transportista
- s: Nivel de servicio
- t: Tiempo de envío [días]
- m: Modo de transporte (aire o tierra)
- q: Cantidad de items en cada pedido [items]
- w: Peso [kg/item]
- F: Peso máximo [kg/item]

- C: Costo de almacenamiento [USD/item]
- M: Costo fijo de transporte [USD/kg]
- X: Costo de almacenamiento [USD]
- Y: Costo de transporte [USD]
- Z: Costo total de la cadena de suministro [USD]

In [5]:
pedidos = Orders.pedidos[["Producto", "Items", "Peso [kg/item]"]]

df1 = pd.merge(pedidos, ProductsPerPlant.productos_por_planta, on = "Producto")
df2 = pd.merge(df1, PlantPorts.puertos_por_planta, on = "Planta")
df3 = pd.merge(df2, Freights.fletes, on = ["Puerto de origen"])
df4 = pd.merge(df3, WhCosts.costos_almacenaje, on = "Planta")
df5 = pd.merge(df4, WhCapacities.capacidad_almacenaje, on = "Planta")
#df6 = pd.merge(df5, Customers.clientes_por_planta, on = "Planta")

df6 = df5

df6.round(2).head()

,Producto,Items,Peso [kg/item],Planta,Puerto de origen,Transportista,Puerto de destino,Peso mínimo [kg/item],Peso máximo [kg/item],Nivel de servicio,Costo mínimo [USD/kg],Tasa de flete [USD/kg],Modo de transporte,Envío [días],Tipo de transportista,Costo específico [USD/item],Capacidad [pedidos/día]
0,1699333,1529,3.01,3,4,V444_8,9,100.0,249.99,DTD,21.47,0.06,Aire,14,V88888888_0,0.52,1013
1,1699333,1529,3.01,3,4,V444_8,9,0.0,99.99,DTD,21.47,0.06,Aire,14,V88888888_0,0.52,1013
2,1699333,1529,3.01,3,4,V444_8,9,500.0,1999.99,DTD,21.47,0.06,Aire,14,V88888888_0,0.52,1013
3,1699333,1529,3.01,3,4,V444_8,9,250.0,499.99,DTD,21.47,0.06,Aire,14,V88888888_0,0.52,1013
4,1699333,1529,3.01,3,4,V444_8,9,2000.0,99999.99,DTD,21.47,0.06,Aire,14,V88888888_0,0.52,1013


*¿La tabla contiene filas duplicadas?*

In [6]:
df6.duplicated().any()

True

*Si la tabla tiene filas duplicadas, estas deben ser eliminadas.*

In [7]:
df6.drop_duplicates(inplace = True)

*¿Cuántas filas y columnas tiene la nueva tabla? (N° filas, N° columnas)*

In [8]:
df6.shape

(3308793, 17)

*Estadística Descriptiva del Costo de Almacenaje [USD]*

$$ X_i = \sum_{k}{q_{ki}*C_i} $$

In [6]:
df6["Costo de almacenaje [USD]"] = df6["Items"] * df6["Costo específico [USD/item]"]

df6["Suma de Costo de almacenaje [USD]"] = df6.groupby(by = "Planta")["Costo de almacenaje [USD]"].transform("sum")

df6.groupby(by = "Planta").agg({"Costo de almacenaje [USD]": "sum"}).rename(columns = {"Costo de almacenaje [USD]": "Suma de Costo de almacenaje [USD]"})

,Suma de Costo de almacenaje [USD]
Planta,
1,3.677521e+05
2,1.806949e+07
3,3.449154e+09
4,4.568211e+07
5,6.921597e+07
6,6.156674e+06
7,2.288741e+07
8,3.306281e+07
9,9.353452e+07


*Estadística Descriptiva del Costo de Transporte [USD]*

$$ \sum_{k}{w_{kpjcstm}} \leq max \ F_{kpjcstm} $$

In [7]:
df6["Peso [kg]"] = df6["Items"] * df6["Peso [kg/item]"]
df6["Peso máximo [kg]"] = df6["Items"] * df6["Peso máximo [kg/item]"]

atributos = ["Puerto de origen", "Puerto de destino", "Transportista", "Modo de transporte", "Envío [días]"]

df6["Suma de Peso [kg]"] = df6.groupby(by = atributos)["Peso [kg]"].transform("sum")
df6["Máximo del Peso máximo [kg]"] = df6.groupby(by = atributos)["Peso máximo [kg]"].transform("max")

df6["Suma de Peso [kg]"] = np.where(df6["Suma de Peso [kg]"] <= df6["Máximo del Peso máximo [kg]"], df6["Suma de Peso [kg]"], df6["Máximo del Peso máximo [kg]"])

df6.head()

,Producto,Items,Peso [kg/item],Planta,Puerto de origen,Transportista,Puerto de destino,Peso mínimo [kg/item],Peso máximo [kg/item],Nivel de servicio,...,Envío [días],Tipo de transportista,Costo específico [USD/item],Capacidad [pedidos/día],Costo de almacenaje [USD],Suma de Costo de almacenaje [USD],Peso [kg],Peso máximo [kg],Suma de Peso [kg],Máximo del Peso máximo [kg]
0,1699333,1529,3.008251,3,4,V444_8,9,100.0,249.99,DTD,...,14,V88888888_0,0.517502,1013,791.260392,3.449154e+09,4599.616432,3.822347e+05,1.978384e+10,5.618469e+10
1,1699333,1529,3.008251,3,4,V444_8,9,0.0,99.99,DTD,...,14,V88888888_0,0.517502,1013,791.260392,3.449154e+09,4599.616432,1.528847e+05,1.978384e+10,5.618469e+10
2,1699333,1529,3.008251,3,4,V444_8,9,500.0,1999.99,DTD,...,14,V88888888_0,0.517502,1013,791.260392,3.449154e+09,4599.616432,3.057985e+06,1.978384e+10,5.618469e+10
3,1699333,1529,3.008251,3,4,V444_8,9,250.0,499.99,DTD,...,14,V88888888_0,0.517502,1013,791.260392,3.449154e+09,4599.616432,7.644847e+05,1.978384e+10,5.618469e+10
4,1699333,1529,3.008251,3,4,V444_8,9,2000.0,99999.99,DTD,...,14,V88888888_0,0.517502,1013,791.260392,3.449154e+09,4599.616432,1.529000e+08,1.978384e+10,5.618469e+10


$$ Si \ s = CRF \ \Rightarrow \ Y_{kpjcstm} = 0 $$

$$ Si \ s \neq CRF, \ m = Tierra \ \Rightarrow \ Y_{kpjcstm} = \frac{w_{kpjcstm}}{\sum_{k}{w_{kpjcstm}}}*R_{kpjcstm} $$

$$ Si \ s \neq CRF, \ m \neq Tierra \ \Rightarrow \ Y_{kpjcstm} = w_{kpjcstm}*R_{kpjcstm} $$

$$ Si \ Y_{kpjcstm} < M_{kpjcstm} \ \Rightarrow \ Y_{kpjcstm} = M_{kpjcstm} $$

In [8]:
a = df6["Peso [kg]"] / df6["Suma de Peso [kg]"] * df6["Tasa de flete [USD/kg]"]
b = df6["Peso [kg]"] * df6["Tasa de flete [USD/kg]"]
c = df6["Costo mínimo [USD/kg]"]

df6["Costo de transporte [USD]"] = np.where(df6["Nivel de servicio"] == "CRF", 0, 
                                            np.where(df6["Modo de transporte"] == "Tierra", np.where(a < c, a, c), np.where(b < c, b, c)))

atributos = ["Puerto de origen", "Puerto de destino"]
df6["Suma de Costo de transporte [USD]"] = df6.groupby(by = atributos)["Costo de transporte [USD]"].transform("sum")

df6.head()

,Producto,Items,Peso [kg/item],Planta,Puerto de origen,Transportista,Puerto de destino,Peso mínimo [kg/item],Peso máximo [kg/item],Nivel de servicio,...,Costo específico [USD/item],Capacidad [pedidos/día],Costo de almacenaje [USD],Suma de Costo de almacenaje [USD],Peso [kg],Peso máximo [kg],Suma de Peso [kg],Máximo del Peso máximo [kg],Costo de transporte [USD],Suma de Costo de transporte [USD]
0,1699333,1529,3.008251,3,4,V444_8,9,100.0,249.99,DTD,...,0.517502,1013,791.260392,3.449154e+09,4599.616432,3.822347e+05,1.978384e+10,5.618469e+10,21.4704,2.355467e+07
1,1699333,1529,3.008251,3,4,V444_8,9,0.0,99.99,DTD,...,0.517502,1013,791.260392,3.449154e+09,4599.616432,1.528847e+05,1.978384e+10,5.618469e+10,21.4704,2.355467e+07
2,1699333,1529,3.008251,3,4,V444_8,9,500.0,1999.99,DTD,...,0.517502,1013,791.260392,3.449154e+09,4599.616432,3.057985e+06,1.978384e+10,5.618469e+10,21.4704,2.355467e+07
3,1699333,1529,3.008251,3,4,V444_8,9,250.0,499.99,DTD,...,0.517502,1013,791.260392,3.449154e+09,4599.616432,7.644847e+05,1.978384e+10,5.618469e+10,21.4704,2.355467e+07
4,1699333,1529,3.008251,3,4,V444_8,9,2000.0,99999.99,DTD,...,0.517502,1013,791.260392,3.449154e+09,4599.616432,1.529000e+08,1.978384e+10,5.618469e+10,21.4704,2.355467e+07


*Estadística Descriptiva del Costo Total [USD]*

$$ Z_{ipj} = min \ (X_{ki} + Y_{kpj}) $$

In [9]:
g6 = df6[["Planta", "Puerto de origen", "Puerto de destino", "Suma de Costo de almacenaje [USD]", "Suma de Costo de transporte [USD]"]].copy()
g60 = g6.drop_duplicates()

g60["Suma de Costo total [USD]"] = g60["Suma de Costo de almacenaje [USD]"] + g60["Suma de Costo de transporte [USD]"]

total = g60["Suma de Costo total [USD]"].sum()
g60["% Suma de Costo Total"] = g60["Suma de Costo total [USD]"].apply(lambda x: x / total * 100)

g60 = g60.set_index(["Planta", "Puerto de origen", "Puerto de destino"])

g60.round(2)

<ipython-input-9-15ef9e22f81b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g60["Suma de Costo total [USD]"] = g60["Suma de Costo de almacenaje [USD]"] + g60["Suma de Costo de transporte [USD]"]
<ipython-input-9-15ef9e22f81b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g60["% Suma de Costo Total"] = g60["Suma de Costo total [USD]"].apply(lambda x: x / total * 100)


,,,Suma de Costo de almacenaje [USD],Suma de Costo de transporte [USD],Suma de Costo total [USD],% Suma de Costo Total
Planta,Puerto de origen,Puerto de destino,,,,
3,4,9,3.449154e+09,23554666.10,3.472709e+09,83.59
9,4,9,9.353452e+07,23554666.10,1.170892e+08,2.82
11,4,9,1.089113e+08,23554666.10,1.324660e+08,3.19
8,4,9,3.306281e+07,23554666.10,5.661748e+07,1.36
12,4,9,1.979798e+07,23554666.10,4.335264e+07,1.04
13,4,9,5.444972e+06,23554666.10,2.899964e+07,0.70
4,5,9,4.568211e+07,1079039.40,4.676115e+07,1.13
2,3,9,1.806949e+07,991752.61,1.906124e+07,0.46
10,2,9,1.250730e+08,4128297.25,1.292013e+08,3.11


*¿Qué conjunto Planta, Puerto de origen y Puerto de destino tiene la menor Suma de Costo Total?*

In [15]:
atributos = ["Planta", "Puerto de origen", "Puerto de destino"]
values = g60["Suma de Costo total [USD]"].idxmin()

dict(list(zip(atributos, values)))

{'Planta': 14, 'Puerto de origen': 7, 'Puerto de destino': 9}